In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MCBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super(MCBAM, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1, bias=False)
        self.sigmoid_channel = nn.Sigmoid()

        self.spatial_conv = nn.Conv2d(3, 1, kernel_size=7, stride=1, padding=3, bias=False)
        self.sigmoid_spatial = nn.Sigmoid()

    def split_channel(self, x):
        H, W = x.size(2), x.size(3)
        h_mid, w_mid = H // 2, W // 2
        return [
            x[:, :, :h_mid, :w_mid],
            x[:, :, :h_mid, w_mid:],
            x[:, :, h_mid:, :w_mid],
            x[:, :, h_mid:, w_mid:]
        ]

    def split_spatial(self, x):
        C = x.size(1)
        split = [C // 3, C // 3, C - 2 * (C // 3)]
        return torch.split(x, split, dim=1)

    def forward(self, x):
        # Channel Attention
        splits = self.split_channel(x)
        channel_att = sum(self.fc2(self.relu(self.fc1(self.avg_pool(s)))) for s in splits)
        channel_att = self.sigmoid_channel(channel_att)
        x = x * channel_att

        # Spatial Attention
        splits = self.split_spatial(x)
        avg_maps = [torch.mean(s, dim=1, keepdim=True) for s in splits]
        spatial_att = self.sigmoid_spatial(self.spatial_conv(torch.cat(avg_maps, dim=1)))
        x = x * spatial_att

        return x


class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=0),
            nn.BatchNorm2d(32),
            MCBAM(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=0),
            nn.BatchNorm2d(64),
            MCBAM(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=0),
            nn.BatchNorm2d(128),
            MCBAM(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(128 * 17 * 17, 10)
        self.fc2 = nn.Linear(10, 4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.sigmoid(self.fc2(x)))
        return x
